##Assignment 1 - enhancement and superresolution



In [ ]:
import numpy as np
import imageio.v3 as imageio
#import matplotlib.pyplot as plt
import sys

"""
Número USP: 11801199
Curso: Engenharia de Computação
Semestre: 2024/1
Título: 1 - Enhancement and superresolution

"""

'\nNúmero USP: 11801199\nCurso: Engenharia de Computação\nSemestre: 2024/1\nTítulo: 1 - Enhancement and superresolution\n\n'

#Basic histogram

In [ ]:
def histogram (img, no_levels):

  # gets the size of the input matrix
  N, M = img.shape

  # creates an empty histogram with size proportional to the number of graylevels
  hist = np.zeros(no_levels).astype(int)

  # computes for all levels in the range
  for i in range(no_levels):
      # sum all positions in which img == i is true
      pixels_value_i = np.sum(img == i)
      # store it in the histogram array
      hist[i] = pixels_value_i

  return(hist)

#Option 1

In [ ]:
def single_img_comulative_histogram(img, no_levels):

  # computes the image histogram
  hist = histogram(img, 256)

  # creates an empty cumulative histogram
  histC = np.zeros(no_levels).astype(int)

  # computes the cumulative histogram
  histC[0] = hist[0] # first value (intensity 0)
  # from intensity 1 to no_levels-1
  for i in range(1,  no_levels):
      histC[i] = hist[i] + histC[i-1]

  # the vector below is used to store the actual transformation function
  # it allows us to later visualize what was the function that computed
  # the equalisation
  hist_transform = np.zeros(no_levels).astype(np.uint8)

  # gets the size of the input image
  N, M = img.shape

  # create the image to store the equalised version
  img_eq = np.zeros([N,M]).astype(np.uint8)

  # for each intensity value, transforms it into a new intensity
  # using the np.where() function
  for z in range(no_levels):
      # computes what would be the output level 's' for an input value 'z'
      s = ((no_levels-1)/float(M*N))*histC[z]

      # for every coordinate in which matrix img has the value 'z'
      # assigns the transformed/equalised value 's'
      img_eq[ np.where(img == z) ] = s

      # store the tranformation function
      hist_transform[z] = s

  return (img_eq)

#Option 2

In [ ]:
def joint_cumulative_histogram(imglow, no_levels):

  # Compute histograms for each image
  h0 = histogram(imglow[0], 256)
  h1 = histogram(imglow[1], 256)
  h2 = histogram(imglow[2], 256)
  h3 = histogram(imglow[3], 256)

  # Combining the histograms
  hist = h0+h1+h2+h3

  #creates an empty cumulative histogram
  histC = np.zeros(no_levels).astype(int)

  # computes the cumulative histogram
  histC[0] = hist[0] # first value (intensity 0)
  # from intensity 1 to no_levels-1
  for i in range(1,  no_levels):
      histC[i] = hist[i] + histC[i-1]

  # the vector below is used to store the actual transformation function
  # it allows us to later visualize what was the function that computed
  # the equalisation
  hist_transform = np.zeros(no_levels).astype(np.uint8)

  # gets the size of the input image
  N, M = img.shape

  # create the image to store the equalised version

  img_eq = [np.zeros([N,M]).astype(np.uint8)]*4

  # for each intensity value, transforms it into a new intensity
  # using the np.where() function
  for j in range(0,3):
    for z in range(no_levels):
        # computes what would be the output level 's' for an input value 'z'
        s = ((no_levels-1)/float(4*M*N))*histC[z]

        # for every coordinate in which matrix img has the value 'z'
        # assigns the transformed/equalised value 's'
        img_eq[j][ np.where(imglow[j] == z) ] = s

        # store the tranformation function
        hist_transform[z] = s

  return (img_eq)

#Option 3

In [ ]:
def gamma_correction(img,gamma):
  # appling the gamma correction formula
  new_img = 255*((img/255.0)**(1/gamma))
  return(new_img).astype('uint8')

#Superresolution

In [ ]:
def superresolution(e_img):
  # Create an empty high-resolution image array with dimensions 512x512
  highimg = np.zeros([512,512])

  # Upscale the low-resolution images and place them in the high-resolution array
  highimg [::2,::2] = e_img[0]  # Upscale and place the first quadrant
  highimg [1::2,::2] = e_img[1]  # Upscale and place the second quadrant
  highimg [::2,1::2] = e_img[2]  # Upscale and place the third quadrant
  highimg [1::2,1::2] = e_img[3]  # Upscale and place the fourth quadrant
  return highimg

#RMSE

In [ ]:
def calculate_rmse(imghigh, h_hat):
  rmse = np.sqrt(np.mean((imghigh-h_hat)**2)) # appling the error formula
  return rmse

#main

In [ ]:
# Getting the inputs
basename = input().strip()
imghigh = input().strip()
F = int(input().strip())
gamma = float(input().strip())

# Loading the images
low_res_images = []
for i in range(0, 4):
        filename = f'{basename}{i}.png'# Compute histograms for each
        img = imageio.imread(filename)
        low_res_images.append(img)
hig_res_images = imageio.imread(imghigh)

# Method selection
if F == 0:
  enhanced_images = low_res_images
elif F == 1:
  enhanced_images = [single_img_comulative_histogram(img, 256) for img in low_res_images]
elif F == 2:
  enhanced_images = joint_cumulative_histogram(low_res_images, 256)
elif F == 3:
  enhanced_images = [gamma_correction(img, gamma) for img in low_res_images]
else:
  print("Error in the input parameter F.")
  sys.exit(1)

# Appling superresolution
H_hat = superresolution(enhanced_images)

# Calculating RMSE
rmse = calculate_rmse(hig_res_images, H_hat)
print(f'{rmse:.4f}')

01_low
01_high.png
2
1
9.2855
